# Fetching Data for Human Features
We have prepared shapefiles containing the USGS quarter quadrangles that have good coverage of forest stand delineations that we want to grab other data for. We'll fetch roads, parcel boundaries, and building footprints from several web services.

Building footprints are generated by Microsoft based on a neural network trained on aerial imagery. 

# Mount Google Drive 
So we can access our files showing tile locations, and save the rasters we will generate from the elevation data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! sudo apt-get install -y libspatialindex-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 21 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (604 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.

In [ ]:
! pip install geopandas rtree overpass -q

     |████████████████████████████████| 972kB 3.3MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 14.8MB 283kB/s 
     |████████████████████████████████| 10.9MB 8.2MB/s 


The following functions will do the work to retrieve our feature layers.

In [ ]:
import numpy as np
import geopandas as gpd
import overpass
import os
import requests
from shapely.geometry import box, Polygon

In [ ]:
def buildings_from_microsoft(bbox, inSR):
    """Returns building footprints generated by Microsoft and hosted by an
    ArcGIS Feature Server.

    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy).
    inSR : int
      spatial reference for bounding box, such as an EPSG code (e.g., 4326)
    """
    BASE_URL = ''.join([
        'https://services.arcgis.com/P3ePLMYs2RVChkJx/ArcGIS/rest/services/',
        'MSBFP2/FeatureServer/0/query?'
    ])

    params = dict(where=None,
                  objectIds=None,
                  time=None,
                  geometry=','.join([str(x) for x in bbox]),
                  geometryType='esriGeometryEnvelope',
                  inSR=inSR,
                  spatialRel='esriSpatialRelIntersects',
                  resultType='none',
                  distance=0.0,
                  units='esriSRUnit_Meter',
                  returnGeodetic='false',
                  outFields=None,
                  returnGeometry='true',
                  returnCentroid='false',
                  featureEncoding='esriDefault',
                  multipatchOption='xyFootprint',
                  maxAllowableOffset=None,
                  geometryPrecision=None,
                  outSR=inSR,
                  datumTransformation=None,
                  applyVCSProjection='false',
                  returnIdsOnly='false',
                  returnUniqueIdsOnly='false',
                  returnCountOnly='false',
                  returnExtentOnly='false',
                  returnQueryGeometry='false',
                  returnDistinctValues='false',
                  cacheHint='false',
                  orderByFields=None,
                  groupByFieldsForStatistics=None,
                  outStatistics=None,
                  having=None,
                  resultOffset=None,
                  resultRecordCount=None,
                  returnZ='false',
                  returnM='false',
                  returnExceededLimitFeatures='true',
                  quantizationParameters=None,
                  sqlFormat='none',
                  f='pgeojson',
                  token=None)

    r = requests.get(BASE_URL, params=params)

    jsn = r.json()

    if len(jsn['features']) == 0:
        gdf = gpd.GeoDataFrame(geometry=[Polygon()], crs=inSR)
    else:
        gdf = gpd.GeoDataFrame.from_features(jsn, crs=inSR)
        gdf = gpd.clip(gdf, box(*bbox))
        if len(gdf) == 0:
            gdf = gpd.GeoDataFrame(geometry=[Polygon()], crs=inSR)
            
    return gdf

In [ ]:
def parcels_from_wa(bbox, inSR=4326, **kwargs):
    """
    Returns tax lot boundaries as features from the Washington Geospatial
    Open Data Portal

    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy).
    inSR : int
      spatial reference for bounding box, such as an EPSG code (e.g., 4326)

    Returns
    -------
    clip_gdf : GeoDataFrame
      features in vector format, clipped to bbox
    """
    BASE_URL = ''.join([
        'https://services.arcgis.com/jsIt88o09Q0r1j8h/arcgis/rest/services/',
        'Current_Parcels_2020/FeatureServer/0/query?'
    ])

    params = dict(
        where=None,
        objectIds=None,
        time=None,
        geometry=','.join([str(x) for x in bbox]),
        geometryType='esriGeometryEnvelope',
        inSR=inSR,
        spatialRel='esriSpatialRelEnvelopeIntersects',
        resultType='none',
        distance=0.0,
        units='esriSRUnit_Meter',
        returnGeodetic='false',
        outFields='*',
        returnGeometry='true',
        returnCentroid='false',
        featureEncoding='esriDefault',
        multipatchOption='xyFootprint',
        maxAllowableOffset=None,
        geometryPrecision=None,
        outSR=inSR,
        datumTransformation=None,
        applyVCSProjection='false',
        returnIdsOnly='false',
        returnUniqueIdsOnly='false',
        returnCountOnly='false',
        returnExtentOnly='false',
        returnQueryGeometry='false',
        returnDistinctValues='false',
        cacheHint='false',
        orderByFields=None,
        groupByFieldsForStatistics=None,
        outStatistics=None,
        having=None,
        resultOffset=None,
        resultRecordCount=None,
        returnZ='false',
        returnM='false',
        returnExceededLimitFeatures='true',
        quantizationParameters=None,
        sqlFormat='none',
        f='pgeojson',
        token=None,
    )
    for key, value in kwargs.items():
        params.update({key: value})

    r = requests.get(BASE_URL, params=params)
    jsn = r.json()
    if len(jsn['features']) == 0:
        clip_gdf = gpd.GeoDataFrame(geometry=[Polygon()], crs=inSR)
    else:
        gdf = gpd.GeoDataFrame.from_features(jsn, crs=inSR)
        gdf['geometry'] = gdf.buffer(0)
        clip_gdf = gpd.clip(gdf, box(*bbox))

    return clip_gdf

In [ ]:
def parcels_from_or(bbox,
                    inSR=4326,
                    **kwargs):
    """
    Returns tax lot boundaries as features from ORMAP

    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy).
    inSR : int
      spatial reference for bounding box, such as an EPSG code (e.g., 4326)

    Returns
    -------
    clip_gdf : GeoDataFrame
      features in vector format, clipped to bbox
    """
    BASE_URL = ''.join([
            'https://utility.arcgis.com/usrsvcs/servers/',
            '78bbb0d0d9c64583ad5371729c496dcc/rest/services/',
            'Secure/DOR_ORMAP/MapServer/3/query?',
    ])

    params = dict(
        f='geojson',
        returnGeometry='true',
        spatialRel='esriSpatialRelIntersects',
        geometry=(f'{{"xmin":{bbox[0]},"ymin":{bbox[1]},'
                  f'"xmax":{bbox[2]},"ymax":{bbox[3]},'
                  f'"spatialReference":{{"wkid":{inSR}}}}}'),
        geometryType='esriGeometryEnvelope',
        outFields='*',
        outSR=inSR
    )
    for key, value in kwargs.items():
        params.update({key: value})

    r = requests.get(BASE_URL, params=params)
    jsn = r.json()
    if len(jsn['features']) == 0:
        clip_gdf = gpd.GeoDataFrame(geometry=[Polygon()], crs=inSR)
    else:
        gdf = gpd.GeoDataFrame.from_features(jsn, crs=inSR)
        gdf['geometry'] = gdf.buffer(0)
        clip_gdf = gpd.clip(gdf, box(*bbox))

    return clip_gdf

In [ ]:
def roads_from_osm(bbox, crs=None):
    """Retrieves ways from Open Street Map clipped to a bounding box.

    Parameters
    ----------
    bbox : list-like
      list of bounding box coordinates (minx, miny, maxx, maxy).
    crs : coordinate reference system
      a string, integer, or class instance which can be interpreted by
      GeoPandas as a Coordinate References System.
    
    Returns
    -------
    gdf : GeoDataFrame
      features in vector format, clipped to bbox
    """
    if crs:
        geom = box(*bbox)
        bbox_gdf = gpd.GeoDataFrame(geometry=[geom], crs=crs)
        latlon_bbox = bbox_gdf.to_crs(epsg=4326).iloc[0]['geometry'].bounds
    else:
        latlon_bbox = box(*bbox)
    
    xmin, ymin, xmax, ymax = latlon_bbox

    query = ''.join(['way["highway"]["highway"!~"cycleway|footway|path|',
                     'pedestrian|steps|corridor|elevator|escalator|proposed|',
                     'construction|bridleway|abandoned|platform|',
                     '["motor_vehicle"!~"no"]["motorcar"!~"no"]',
                     f'({ymin},{xmin},{ymax},{xmax});(._;>;)'])

    api = overpass.API()
    records = api.get(query, verbosity='geom')
    gdf = gpd.GeoDataFrame.from_features(records)
    if len(gdf) > 0:
        gdf = gdf.loc[gdf.geometry.type.isin(['LineString', 'MultiLineString'])]
        gdf = gpd.clip(gdf, box(*latlon_bbox))
        gdf.crs = 4326
        if crs:
            gdf = gdf.to_crs(crs)

    else:
        gdf = gpd.GeoDataFrame(geometry=[Polygon()], crs=crs)

    return gdf

# Download Data for Training Tiles


These functions will loop through a GeoDataFrame, fetch the relevant data, and write them to disk in the appropriate formats.

In [ ]:
def fetch_buildings(gdf, state, overwrite=False):
    epsg = gdf.crs.to_epsg()
    print('Fetching building footprints for {:,d} tiles'.format(len(gdf)))

    ## loop through all the geometries in the geodataframe and fetch NLCD data

    for idx, row in gdf.iterrows():
        xmin, ymin, xmax, ymax = row['geometry'].bounds
        xmin, ymin = np.floor((xmin, ymin))
        xmax, ymax = np.ceil((xmax, ymax))

        bbox = [xmin, ymin, xmax, ymax]

        ## don't bother fetching data if we already have processed this tile
        OUTROOT = '/content/drive/Shared drives/stand_mapping/data/interim/training_tiles'
        outfolder = f'{state.lower()}/buildings'
        outdir = os.path.join(OUTROOT, outfolder)

        outname = f'{row.CELL_ID}_buildings.geojson'
        outfile = os.path.join(outdir, outname)        
        if os.path.exists(outfile) and not overwrite:
            if idx % 100 == 0:
                print()
            if idx % 10 == 0:
                print(idx, end='')
            else:
                print('.', end='')
            continue
        
        buildings = buildings_from_microsoft(bbox, epsg)
        buildings.to_file(outfile, driver='GeoJSON')
        
        ## report progress
        if idx % 100 == 0:
            print()
        if idx % 10 == 0:
            print(idx, end='')
        else:
            print('.', end='')

In [ ]:
def fetch_parcels(gdf, state, overwrite=False):
    epsg = gdf.crs.to_epsg()
    print('Fetching parcel boundaries for {:,d} tiles'.format(len(gdf)))

    ## loop through all the geometries in the geodataframe and fetch NLCD data

    for idx, row in gdf.iterrows():
        xmin, ymin, xmax, ymax = row['geometry'].bounds
        xmin, ymin = np.floor((xmin, ymin))
        xmax, ymax = np.ceil((xmax, ymax))

        bbox = [xmin, ymin, xmax, ymax]

        OUTROOT = '/content/drive/Shared drives/stand_mapping/data/interim/training_tiles'
        outfolder = f'{state.lower()}/parcels'
        outdir = os.path.join(OUTROOT, outfolder)

        outname = f'{row.CELL_ID}_parcels.geojson'
        outfile = os.path.join(outdir, outname)        
        if os.path.exists(outfile) and not overwrite:
            if idx % 100 == 0:
                print()
            if idx % 10 == 0:
                print(idx, end='')
            else:
                print('.', end='')
            continue
        
        if state.lower() == 'washington':
            parcels = parcels_from_wa(bbox, epsg)
        elif state.lower() == 'oregon':
            parcels = parcels_from_or(bbox, epsg)
        
        parcels.to_file(outfile, driver='GeoJSON')
        
        ## report progress
        if idx % 100 == 0:
            print()
        if idx % 10 == 0:
            print(idx, end='')
        else:
            print('.', end='')

In [ ]:
def fetch_roads(gdf, state, overwrite=False):
    crs = gdf.crs
    print('Fetching roads for {:,d} tiles'.format(len(gdf)))

    ## loop through all the geometries in the geodataframe and fetch roads

    for idx, row in gdf.iterrows():
        xmin, ymin, xmax, ymax = row['geometry'].bounds
        xmin, ymin = np.floor((xmin, ymin))
        xmax, ymax = np.ceil((xmax, ymax))

        bbox = [xmin, ymin, xmax, ymax]

        ## don't bother fetching data if we already have processed this tile
        OUTROOT = '/content/drive/Shared drives/stand_mapping/data/interim/training_tiles'
        outfolder = f'{state.lower()}/roads'
        outdir = os.path.join(OUTROOT, outfolder)

        outname = f'{row.CELL_ID}_roads.geojson'
        outfile = os.path.join(outdir, outname)        
        if os.path.exists(outfile) and not overwrite:
            if idx % 100 == 0:
                print()
            if idx % 10 == 0:
                print(idx, end='')
            else:
                print('.', end='')
            continue
        
        roads = roads_from_osm(bbox, crs)
        try:
            roads.to_file(outfile, driver='GeoJSON')
        except:
            print(outfile)
        
        ## report progress
        if idx % 100 == 0:
            print()
        if idx % 10 == 0:
            print(idx, end='')
        else:
            print('.', end='')

In [ ]:
SHP_DIR = '/content/drive/Shared drives/stand_mapping/data/interim'

WA11_SHP = 'washington_utm11n_training_quads_epsg6340.shp'
WA10_SHP = 'washington_utm10n_training_quads_epsg6339.shp'
OR10_SHP = 'oregon_utm10n_training_quads_epsg6339.shp'
OR11_SHP = 'oregon_utm11n_training_quads_epsg6340.shp'

or10_gdf = gpd.read_file(os.path.join(SHP_DIR, OR10_SHP))
or11_gdf = gpd.read_file(os.path.join(SHP_DIR, OR11_SHP))
wa10_gdf = gpd.read_file(os.path.join(SHP_DIR, WA10_SHP))
wa11_gdf = gpd.read_file(os.path.join(SHP_DIR, WA11_SHP))

## Fetch building footprints for each tile

In [ ]:
GDF = or11_gdf
STATE = 'oregon'

fetch_buildings(GDF, STATE)

Fetching building footprints for 524 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........
300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........
400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........
500.........510.........520...

In [ ]:
GDF = or10_gdf
STATE = 'oregon'

fetch_buildings(GDF, STATE)

Fetching building footprints for 607 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........
300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........
400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........
500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........
600......

In [ ]:
GDF = wa10_gdf
STATE = 'washington'

fetch_buildings(GDF, STATE)

Fetching building footprints for 277 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270......

In [ ]:
GDF = wa11_gdf
STATE = 'washington'

fetch_buildings(GDF, STATE)

Fetching building footprints for 82 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.

## Fetch parcel boundaries for each tile

In [ ]:
GDF = wa10_gdf
STATE = 'washington'

fetch_parcels(GDF, STATE)

Fetching parcel boundaries for 277 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270......

In [ ]:
GDF = wa11_gdf
STATE = 'washington'

fetch_parcels(GDF, STATE)

Fetching parcel boundaries for 82 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.

In [ ]:
GDF = or11_gdf
STATE = 'oregon'

fetch_parcels(GDF, STATE)

Fetching parcel boundaries for 524 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........
300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........
400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........
500.........510.........520...

In [ ]:
GDF = or10_gdf
STATE = 'oregon'

fetch_parcels(GDF, STATE)

Fetching parcel boundaries for 607 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........
300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........
400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........
500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........
600......

## Fetch roads for each tile

In [ ]:
GDF = wa11_gdf
STATE = 'washington'

fetch_roads(GDF, STATE)

Fetching roads for 82 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.

In [ ]:
GDF = wa10_gdf
STATE = 'washington'

fetch_roads(GDF, STATE)

Fetching roads for 277 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270......

In [ ]:
GDF = or10_gdf
STATE = 'oregon'

fetch_roads(GDF, STATE)

Fetching roads for 607 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........
300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........
400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........
500.........510.........520.........530.........540.........550.........560.........570.........580.........590.........
600......

In [ ]:
GDF = or11_gdf
STATE = 'oregon'

fetch_roads(GDF, STATE)

Fetching roads for 524 tiles

0.........10.........20.........30.........40.........50.........60.........70.........80.........90.........
100.........110.........120.........130.........140.........150.........160.........170.........180.........190.........
200.........210.........220.........230.........240.........250.........260.........270.........280.........290.........
300.........310.........320.........330.........340.........350.........360.........370.........380.........390.........
400.........410.........420.........430.........440.........450.........460.........470.........480.........490.........
500.........510.........520...